In [1]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from fromage import models
from visual_dialog import dialog_utils

dialogs_path = 'visual_dialog/visdial_1.0_val.json'
annotations_path = 'visual_dialog/visdial_1.0_val_dense_annotations.json'
images_path = 'visual_dialog/VisualDialog_val2018/'

In [2]:
# Read the JSON file and load it into a Python object
with open(dialogs_path, 'r') as f:
    dialogs = json.load(f)

answer_index = dialogs['data']['dialogs'][0]['dialog'][0]['answer']
question_index = dialogs['data']['dialogs'][0]['dialog'][0]['question']
image_id = dialogs['data']['dialogs'][0]['image_id']

question = dialogs['data']['questions'][question_index]
answer = dialogs['data']['answers'][answer_index]

print(question)
print(answer)

is the photo in color
yes


In [3]:
# Get image file by image id

def get_image(image_id, show_image=False):
    # Loop through all the files in the folder
    for file_name in os.listdir(images_path):
        # Check if the file name ends with the number and .jpg suffix
        if file_name.endswith(str(image_id) + '.jpg'):
            # Create the full path to the matching file
            # Do something with the file, e.g. print its path
            image_path = os.path.join(images_path, file_name)
            image = Image.open(image_path)
            if show_image:
                plt.figure(figsize=(3, 3))
                plt.axis('off')
                plt.imshow(np.array(image))
                plt.show()
            return image


In [4]:
# Load model used in the paper.
# model_dir = './fromage_model/'
# model = models.load_fromage(model_dir)

In [5]:
# CREATE DIALOG DATAFRAME

def create_stories(dialogs_path:str) -> pd.DataFrame:

    # Read the JSON file and load it into a Python object
    with open(dialogs_path, 'r') as f:
        dialogs = json.load(f)

    dialog_df = pd.DataFrame(columns=['id', 'round', 'image_id', 'question_id', 'question', 'answer_id', 'answer'])

    # LOAD STORY-IN-SEQUENCE JSON
    for i in range(len(dialogs['data']['dialogs'])):
        id = i

        image_id = dialogs['data']['dialogs'][i]['image_id']
        # get_image(image_id)

        for j in range(len(dialogs['data']['dialogs'][i]['dialog'])):
                round = j+1
                answer_index = dialogs['data']['dialogs'][i]['dialog'][j]['answer']
                question_index = dialogs['data']['dialogs'][i]['dialog'][j]['question']

                question = dialogs['data']['questions'][question_index]
                answer = dialogs['data']['answers'][answer_index]

                new_row = {'id': id, 'round': round, 'image_id': image_id, 'question_id': question_index, 
                           'question': question, 'answer_id': answer_index, 'answer': answer}

                # Append the new row to the DataFrame using the loc method
                dialog_df.loc[len(dialog_df)] = new_row
    
    return dialog_df


In [6]:
dialogs_path = 'visual_dialog/visdial_1.0_val.json'

dialog_df = create_stories(dialogs_path)

In [ ]:
stories_csv_path = 'visual_dialog/dialogs.csv'
dialog_utils.save_stories(dialog_df, stories_csv_path)

# Load Dialogs Dataframe

In [ ]:
def load_dialogs(stories_csv_path: str) -> pd.DataFrame:
    return pd.read_csv(stories_csv_path, encoding='utf8', dtype=str)

In [ ]:
dialogs_csv_path = 'visual_dialog/dialogs.csv'

dialogs_df = load_dialogs(dialogs_csv_path)

In [ ]:
dialogs_df.head()

In [1]:
# Create prompts from dataframe
def get_prompt_list(dialogs_df):

    prompt, prompt_list = [], []

    for i in range(len(dialogs_df)-1):
        if dialogs_df['round'][i] == 1:
            image_id = dialogs_df['image_id'][i]
            img = get_image(image_id)
            prompt.append(img)
        if dialogs_df['id'][i+1] == dialogs_df['id'][i]:
            prompt.append(dialogs_df['question'][i])
            prompt.append(dialogs_df['answer'][i])
        else:
            prompt_list.append(prompt)
            # print(prompt)
            prompt = []
    return prompt_list

NameError: name 'dialogs_df' is not defined